In [ ]:
# %% [markdown]
# # 4. Jobs Data: RAIS & Economic Geography
#
# **The Problem:** RAIS data is strictly tabular. It tracks formal employment ties but doesn't map them.
#
# **The Solution:** AtlasBR uses the `cep` (Postal Code) of the establishment to place it on the map. This allows us to visualize **Economic Density**.
#
# **What you will learn:**
# 1.  **Mass Geocoding:** Mapping thousands of companies instantly.
# 2.  **Job Hubs:** Identifying where the jobs actually *are* versus where people live.

# %%
import os
import atlasbr
from atlasbr.app.rais import load_rais

atlasbr.configure_logging()
MUNICIPALITY = "Belo Horizonte, MG"

# %% [markdown]
# ## 1. Fetching the "Job Map"
# We load RAIS establishments for 2021.
# *Note: This dataset is large. The CEP join might take a few seconds.*

# %%
gdf_jobs = load_rais(
    places=[MUNICIPALITY],
    year=2021,
    geocode=True, # Essential for spatial analysis
    gcp_billing=os.getenv("GOOGLE_CLOUD_PROJECT")
)

print(f"💼 Loaded {len(gdf_jobs)} active establishments.")

# %% [markdown]
# ## 2. Who is hiring?
# Using CNAE (Economic Activity) codes to see the city's profile.

# %%
# Top 5 sectors by number of establishments
top_sectors = gdf_jobs["cnae_2"].value_counts().head(5)
print("Top Economic Sectors:")
print(top_sectors)

# %% [markdown]
# ## 3. Interactive Map: Employment Density
# Instead of mapping every corner store, let's filter for **Large Employers** (> 50 employees).
# This visualizes the Central Business District (CBD) and industrial zones.

# %%
large_employers = gdf_jobs[gdf_jobs["quantidade_vinculos_ativos"] > 50].copy()

print(f"Mapping {len(large_employers)} large employers...")

large_employers.explore(
    column="quantidade_vinculos_ativos",
    cmap="inferno_r", # Hot colors for high job counts
    marker_kwds={"radius": 8},
    scheme="quantiles",
    tooltip=[
        "id_estabelecimento", 
        "cnae_2", 
        "quantidade_vinculos_ativos"
    ],
    tiles="CartoDB DarkMatter",
    legend_kwds={"caption": "Number of Active Jobs"}
)